In [56]:
from gurobipy import Model, GRB, quicksum

In [57]:
# Create a new model
model = Model("Coffee Procurement for Summit")

In [58]:
# Parameters: Scenarios, probabilities, and demand
scenarios = list(range(1, 17))
probabilities = [0.09, 0.12, 0.10, 0.05, 0.16, 0.14, 0.03, 0.08, 0.05, 0.05, 0.04, 0.03, 0.02, 0.01, 0.02, 0.01]
demands = [90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165]

In [59]:
# Decision variables
x = model.addVar(vtype=GRB.CONTINUOUS, name="gallons_primary")  # Gallons from primary supplier
y_phil = model.addVars(scenarios, vtype=GRB.CONTINUOUS, name="gallons_phil")  # Phil & Sebastian
y_rosso = model.addVars(scenarios, vtype=GRB.CONTINUOUS, name="gallons_rosso")  # Rosso Coffee
y_mono = model.addVars(scenarios, vtype=GRB.CONTINUOUS, name="gallons_mono")  # Monogram Coffee
z_rosso = model.addVars(scenarios, vtype=GRB.BINARY, name="order_rosso")  # Binary for Rosso orders
z_mono = model.addVars(scenarios, vtype=GRB.BINARY, name="order_mono")  # Binary for Monogram orders

In [60]:
# Objective function: Minimize total expected cost
model.setObjective(
    95 * x +
    quicksum(probabilities[n-1] * (
        120 * y_phil[n] +
        105 * y_rosso[n] +
        110 * y_mono[n]
    ) for n in scenarios),
    GRB.MINIMIZE)

In [61]:
# Constraints
# Ensure demand is met in each scenario
for n in scenarios:
    model.addConstr(x + y_phil[n] + y_rosso[n] + y_mono[n] >= demands[n-1], f"Demand_met_{n}")

# Minimum order constraints
for n in scenarios:
    model.addConstr(y_rosso[n] >= 70 * z_rosso[n], f"Min_order_rosso_{n}")
    model.addConstr(y_rosso[n] <= 1000 * z_rosso[n], f"Max_order_possible_rosso_{n}")  # 1000 is a sufficiently large number
    model.addConstr(y_mono[n] >= 40 * z_mono[n], f"Min_order_mono_{n}")
    model.addConstr(y_mono[n] <= 1000 * z_mono[n], f"Max_order_possible_mono_{n}")  # Assume 1000 gallons is a large enough cap

In [62]:
# Optimize model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 80 rows, 81 columns and 192 nonzeros
Model fingerprint: 0x3c052ba0
Variable types: 49 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 2e+02]
Found heuristic solution: objective 13758.000000
Presolve time: 0.00s
Presolved: 80 rows, 81 columns, 192 nonzeros
Variable types: 49 continuous, 32 integer (32 binary)

Root relaxation: objective 1.113550e+04, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 11135.5000    0   13 13758.000

In [63]:
# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal gallons of primary coffee to order:", x.x)
    print("Optimal total cost:", model.objVal)
    for n in scenarios:
        print(f"Scenario {n}: Phil {y_phil[n].x}, Rosso {y_rosso[n].x}, Monogram {y_mono[n].x}")
else:
    print("No optimal solution found.")

Optimal gallons of primary coffee to order: 95.0
Optimal total cost: 11343.5
Scenario 1: Phil 0.0, Rosso 0.0, Monogram 0.0
Scenario 2: Phil 0.0, Rosso 0.0, Monogram 0.0
Scenario 3: Phil 5.0, Rosso 0.0, Monogram 0.0
Scenario 4: Phil 10.0, Rosso 0.0, Monogram 0.0
Scenario 5: Phil 15.0, Rosso 0.0, Monogram 0.0
Scenario 6: Phil 20.0, Rosso 0.0, Monogram 0.0
Scenario 7: Phil 25.0, Rosso 0.0, Monogram 0.0
Scenario 8: Phil 30.0, Rosso 0.0, Monogram 0.0
Scenario 9: Phil 35.0, Rosso 0.0, Monogram 0.0
Scenario 10: Phil 0.0, Rosso 0.0, Monogram 40.0
Scenario 11: Phil 0.0, Rosso 0.0, Monogram 45.0
Scenario 12: Phil 0.0, Rosso 0.0, Monogram 50.0
Scenario 13: Phil 0.0, Rosso 0.0, Monogram 55.0
Scenario 14: Phil 0.0, Rosso 0.0, Monogram 60.0
Scenario 15: Phil 0.0, Rosso 0.0, Monogram 65.0
Scenario 16: Phil 0.0, Rosso 70.0, Monogram 0.0


# G) and H)

In [64]:
from gurobipy import Model, GRB, quicksum

def calculate_break_even_no_order_in_advance(demands, probabilities, prices_local):
    model = Model("BreakEvenNoAdvanceOrder")
    model.setParam('OutputFlag', 0)  # Turn off Gurobi output

    # Variables for the amount of coffee ordered from each local supplier in each scenario
    coffee_ordered = model.addVars(len(demands), len(prices_local), lb=0, vtype=GRB.CONTINUOUS, name="CoffeeOrdered")
    
    # Objective: Minimize the expected cost of on-demand procurement
    total_cost = quicksum(probabilities[i] * quicksum(coffee_ordered[i, j] * prices_local[j] for j in range(len(prices_local))) for i in range(len(demands)))
    model.setObjective(total_cost, GRB.MINIMIZE)

    # Constraints: Ensure each scenario's demand is met
    for i in range(len(demands)):
        model.addConstr(quicksum(coffee_ordered[i, j] for j in range(len(prices_local))) == demands[i], name=f"DemandMet_{i}")
    
    model.optimize()
    
    # Extract the minimum cost from the model and calculate break-even price
    if model.status == GRB.OPTIMAL:
        min_cost = model.objVal
        max_demand = max(demands)
        break_even_price = min_cost / max_demand
        print(f"Break-even price per gallon to not order in advance: ${break_even_price:.2f}")
        return break_even_price  # Ensure this is returned
    else:
        print("No optimal solution found.")
        return None

# Example data for testing
demands = [90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165]
probabilities = [0.09, 0.12, 0.10, 0.05, 0.16, 0.14, 0.03, 0.08, 0.05, 0.05, 0.04, 0.03, 0.02, 0.01, 0.02, 0.01]
prices_local = [120, 105, 110]  # Local suppliers' prices

break_even_price = calculate_break_even_no_order_in_advance(demands, probabilities, prices_local)

Break-even price per gallon to not order in advance: $72.96


# I) and J)

In [65]:
from gurobipy import Model, GRB, quicksum

# Hypothetical setup (adjust variables, objective and constraints to fit actual problem details)
demands = [90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165]
probabilities = [0.09, 0.12, 0.10, 0.05, 0.16, 0.14, 0.03, 0.08, 0.05, 0.05, 0.04, 0.03, 0.02, 0.01, 0.02, 0.01]
prices = [95, 120, 105, 110]  # Example prices for primary and emergency suppliers

# Function to solve the stochastic model
def solve_stochastic_model():
    model = Model("Stochastic")
    x = model.addVar(vtype=GRB.CONTINUOUS, name="x_primary")  # Primary coffee order
    y = model.addVars(len(demands), vtype=GRB.CONTINUOUS, name="y_on_demand")  # On-demand orders
    model.setObjective(95 * x + quicksum(probabilities[i] * prices[1] * y[i] for i in range(len(demands))), GRB.MINIMIZE)
    for i in range(len(demands)):
        model.addConstr(x + y[i] >= demands[i])
    model.optimize()
    return model.ObjVal

# Function to calculate WS and EVPI
def calculate_ws_and_evpi():
    ws_costs = []
    for demand, prob in zip(demands, probabilities):
        model = Model("Perfect_Foresight")
        x = model.addVar(vtype=GRB.CONTINUOUS, name="x")
        model.setObjective(prices[0] * x, GRB.MINIMIZE)
        model.addConstr(x >= demand)
        model.optimize()
        ws_costs.append(prob * model.ObjVal)
    ws = sum(ws_costs)
    stochastic_cost = solve_stochastic_model()
    evpi = stochastic_cost - ws
    return ws, evpi

# Function to calculate EEV and VSS
def calculate_eev_and_vss():
    average_demand = sum(p * d for p, d in zip(probabilities, demands))
    model = Model("Mean_Value")
    x = model.addVar(vtype=GRB.CONTINUOUS, name="x")
    model.setObjective(prices[0] * x, GRB.MINIMIZE)
    model.addConstr(x >= average_demand)
    model.optimize()
    eev = model.ObjVal
    stochastic_cost = solve_stochastic_model()
    vss = eev - stochastic_cost
    return eev, vss

# Calculate WS and EVPI
ws, evpi = calculate_ws_and_evpi()
print("Expected Value with Perfect Foresight (WS):", ws)
print("Expected Value of Perfect Information (EVPI):", evpi)

# Calculate EEV and VSS
eev, vss = calculate_eev_and_vss()
print("Expected Value from the Mean Value Problem (EEV):", eev)
print("Value of the Stochastic Solution (VSS):", vss)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 1 rows, 1 columns and 1 nonzeros
Model fingerprint: 0x592e56f5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 9e+01]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.5500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  8.550000000e+03
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical